<a href="https://colab.research.google.com/github/lewirbi/GapfillingSentinel-2/blob/main/satellite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=FDGfedu5O8YSs9tpTHoRxDu56OCKpOpmaqdQqaSlDxQ&tc=pJXZSZ5pO5pMWjhEXfTATHS3S6HjzWb1QJ1BDxUz4Pg&cc=R3Q4ZDexss8UfENVqwK2v3eGcRVOufMHI8dFyYpo41I

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhXVmmJs_y7_V8Fg3AQlk7M8tzLdjlau6mtvXo5TMexqKmGh4iayl34

Successfully saved authorization token.


In [ ]:
# Initialize the library.
ee.Initialize()


In [ ]:
!pip install geemap

In [ ]:
import geemap
import ee
import folium
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from skimage.transform import resize
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten

In [ ]:
# Define the region of interest (Maroua region)
roi = ee.Geometry.Rectangle([14.275, 10.520, 14.605, 10.680])

In [ ]:
# Define the date range for image acquisition
start_date = '2005-01-01'
end_date = '2023-06-01'


In [ ]:
# Filter Sentinel-2 images based on the region and date range
sentinel2_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)

In [ ]:
# Create a map and center it around the region of interest
Map = geemap.Map(center=[10.5881, 14.3359], zoom=10)

In [ ]:
# Add the Sentinel-2 collection to the map
Map.addLayer(sentinel2_collection, {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}, 'Sentinel-2')

In [ ]:
# Display the map
Map


Map(center=[10.5881, 14.3359], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [ ]:
AOI = ee.Geometry.Point(14.275, 10.520)
START_DATE = '2022-01-01'
END_DATE = '2023-06-30'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [ ]:
# Define cloud masking functions
def cloud_mask(aoi, start_date, end_date):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
                 .filterBounds(aoi)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
                        .filterBounds(aoi)
                        .filterDate(start_date, end_date))

    return ee.ImageCollection(
        ee.Join.saveFirst('s2cloudless')
        .apply(primary=s2_sr_col, secondary=s2_cloudless_col,
               condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))
    )

In [ ]:
cloud_mask_calculation = cloud_mask(AOI, START_DATE, END_DATE)

In [ ]:
def bands_clouds_calc(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [ ]:

def bands_shadow_calc(img):
    not_water = img.select('SCL').neq(6)
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH * SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))
    cld_proj = (img.select('clouds')
                .directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST * 10)
                .reproject(crs=img.select(0).projection(), scale=100)
                .select('distance')
                .mask()
                .rename('cloud_transform'))
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [ ]:
def shadow_mask(img):
    img_cloud = bands_clouds_calc(img)
    img_cloud_shadow = bands_shadow_calc(img_cloud)
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER * 2 / 20)
                   .reproject(crs=img.select([0]).projection(), scale=20)
                   .rename('cloudmask'))
    return img_cloud_shadow.addBands(is_cld_shdw)

In [ ]:


def visualize_map(image_collection):
    # Mosaic the image collection.
    img = image_collection.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Define the Earth Engine layer adding function.
    def add_earth_engine_layer_to_map(ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
        map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
        folium.raster_layers.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
            name=name,
            show=show,
            opacity=opacity,
            min_zoom=min_zoom,
            overlay=True,
            control=True
        ).add_to(m)

    # Add layers to the folium map.
    add_earth_engine_layer_to_map(img,
                                  {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                                  'S2 image', True, 1, 9)
    add_earth_engine_layer_to_map(probability,
                                  {'min': 0, 'max': 100},
                                  'probability (cloud)', False, 1, 9)
    add_earth_engine_layer_to_map(clouds,
                                  {'palette': 'e056fd'},
                                  'clouds', False, 1, 9)
    add_earth_engine_layer_to_map(cloud_transform,
                                  {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                                  'cloud_transform', False, 1, 9)
    add_earth_engine_layer_to_map(dark_pixels,
                                  {'palette': 'orange'},
                                  'dark_pixels', False, 1, 9)
    add_earth_engine_layer_to_map(shadows,
                                  {'palette': 'yellow'},
                                  'shadows', False, 1, 9)
    add_earth_engine_layer_to_map(cloudmask,
                                  {'palette': 'orange'},
                                  'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)


In [ ]:
cloud_mask_display = cloud_mask_calculation.map(shadow_mask)

visualize_map(cloud_mask_display)

In [ ]:
cloud_prob_calculation = cloud_mask(AOI, START_DATE, END_DATE)

In [ ]:
def cloudless_calc(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [ ]:
cloud_less_prob = (cloud_prob_calculation.map(shadow_mask)
                             .map(cloudless_calc)
                             .median())

In [ ]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_earth_engine_layer_to_map(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_earth_engine_layer_to_map = add_earth_engine_layer_to_map

In [ ]:
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_earth_engine_layer_to_map(cloud_less_prob,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [ ]:



# Define labeling and image splitting functions
def label_cloud(image):
    return image.set('label', 1)

def label_non_cloud(image):
    return image.set('label', 0)

def split_dataset(image_list, label_list, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(
        image_list, label_list, test_size=test_size, stratify=label_list, random_state=42
    )

    return X_train, X_test, y_train, y_test



# Define feature extraction function
# Define feature extraction functio# Define feature extraction function
def extract_features(image):
    resized_image = image.resample('bilinear').reproject(crs=image.select(0).projection(), scale=20)
    img_dict = resized_image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']).reduceRegion(
        reducer=ee.Reducer.toList(), geometry=AOI, scale=10
    ).getInfo()
    img_values = list(img_dict.values())

    # Remove non-numeric values (e.g., null or NaN)
    img_values = np.array(img_values, dtype=np.float64)
    img_values = img_values[~np.isnan(img_values)]

    # Reshape and normalize the array
    img_array = img_values.reshape((-1, 3))
    img_array = img_array / 2500.0  # Adjust the normalization factor based on the image data range
    return img_array




# Perform labeling
labeled_images = cloud_prob_calculation.map(label_cloud).merge(cloud_prob_calculation.map(label_non_cloud))

# Extract image IDs and labels
image_list = []
label_list = []
for i in range(labeled_images.size().getInfo()):
    image = ee.Image(labeled_images.toList(labeled_images.size()).get(i))
    image_list.append(extract_features(image))
    label_list.append(image.get('label').getInfo())






In [ ]:
# Perform image splitting
X_train, X_test, y_train, y_test = split_dataset(image_list, label_list)

# Resize the images to a common size
resized_X_train = [resize(image, (256, 256)) for image in X_train]
resized_X_test = [resize(image, (256, 256)) for image in X_test]

# Convert the resized images to numpy arrays and flatten
X_train_flattened = np.array(resized_X_train).reshape(len(X_train), -1)
X_test_flattened = np.array(resized_X_test).reshape(len(X_test), -1)
# Train a Random Forest classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_flattened, y_train)

# Evaluate the model
accuracy = classifier.score(X_test_flattened, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.09375


In [ ]:


# Perform image splitting
X_train, X_test, y_train, y_test = split_dataset(image_list, label_list)

# Resize the images to a common size
resized_X_train = [resize(image, (256, 256)) for image in X_train]
resized_X_test = [resize(image, (256, 256)) for image in X_test]

# Convert grayscale to RGB
X_train_rgb = [np.stack((image,) * 3, axis=-1) for image in resized_X_train]
X_test_rgb = [np.stack((image,) * 3, axis=-1) for image in resized_X_test]

# Extract features using a pre-trained CNN
input_shape = (256, 256, 3)
input_tensor = Input(shape=input_shape)
pretrained_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
X_train_features = pretrained_model.predict(np.array(X_train_rgb))
X_test_features = pretrained_model.predict(np.array(X_test_rgb))

# Flatten the feature maps
X_train_flattened = X_train_features.reshape(X_train_features.shape[0], -1)
X_test_flattened = X_test_features.reshape(X_test_features.shape[0], -1)

# Train a Random Forest classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_flattened, y_train)

# Predict labels
y_pred = classifier.predict(X_test_flattened)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


1/1 [==============================] - 26s 26s/step
Accuracy: 0.09375


In [ ]:


# Perform image splitting
X_train, X_test, y_train, y_test = split_dataset(image_list, label_list)

# Resize the images to a common size
resized_X_train = [resize(image, (256, 256)) for image in X_train]
resized_X_test = [resize(image, (256, 256)) for image in X_test]

# Convert grayscale to RGB
X_train_rgb = [np.stack((image,) * 3, axis=-1) for image in resized_X_train]
X_test_rgb = [np.stack((image,) * 3, axis=-1) for image in resized_X_test]

# Extract features using a pre-trained CNN
input_shape = (256, 256, 3)
input_tensor = Input(shape=input_shape)
pretrained_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the pre-trained layers
for layer in pretrained_model.layers:
    layer.trainable = False

# Add custom layers on top
x = pretrained_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=pretrained_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(np.array(X_train_rgb), np.array(y_train), epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(np.array(X_test_rgb))
y_pred_binary = np.round(y_pred).flatten()
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy}")


Epoch 1/10
4/4 [==============================] - 102s 25s/step - loss: 1.0070 - accuracy: 0.5152 - val_loss: 1.7120 - val_accuracy: 0.3600
Epoch 2/10
4/4 [==============================] - 101s 25s/step - loss: 2.2830 - accuracy: 0.5051 - val_loss: 0.7737 - val_accuracy: 0.6400
Epoch 3/10
4/4 [==============================] - 119s 31s/step - loss: 2.8197 - accuracy: 0.5354 - val_loss: 1.1221 - val_accuracy: 0.3600
Epoch 4/10
4/4 [==============================] - 120s 32s/step - loss: 2.5974 - accuracy: 0.5051 - val_loss: 0.7348 - val_accuracy: 0.3600
Epoch 5/10
4/4 [==============================] - 120s 32s/step - loss: 1.9446 - accuracy: 0.4949 - val_loss: 0.6516 - val_accuracy: 0.6400
Epoch 6/10
4/4 [==============================] - 100s 25s/step - loss: 2.6881 - accuracy: 0.4141 - val_loss: 0.6508 - val_accuracy: 0.6400
Epoch 7/10
4/4 [==============================] - 119s 31s/step - loss: 1.8064 - accuracy: 0.5455 - val_loss: 0.7181 - val_accuracy: 0.3600
Epoch 8/10
4/4 [====

In [ ]:


# Perform image splitting
X_train, X_test, y_train, y_test = split_dataset(image_list, label_list)

# Resize the images to a common size
resized_X_train = [resize(image, (256, 256)) for image in X_train]
resized_X_test = [resize(image, (256, 256)) for image in X_test]

# Convert grayscale to RGB
X_train_rgb = [np.stack((image,) * 3, axis=-1) for image in resized_X_train]
X_test_rgb = [np.stack((image,) * 3, axis=-1) for image in resized_X_test]

# Split the training data into training and validation sets
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_rgb, y_train, test_size=0.2, random_state=42
)

# Create data generators with data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(np.array(X_train_final))

# Extract features using a pre-trained CNN
input_shape = (256, 256, 3)
input_tensor = Input(shape=input_shape)
pretrained_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the pre-trained layers
for layer in pretrained_model.layers:
    layer.trainable = False

# Add custom layers on top
x = pretrained_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=pretrained_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(datagen.flow(np.array(X_train_final), np.array(y_train_final), batch_size=32),
                    steps_per_epoch=len(X_train_final) // 32,
                    epochs=10,
                    validation_data=(np.array(X_val), np.array(y_val)),
                    callbacks=[reduce_lr, early_stop])

# Evaluate the model
y_pred = model.predict(np.array(X_test_rgb))
y_pred_binary = np.round(y_pred).flatten()
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy}")


Epoch 1/10
3/3 [==============================] - 81s 25s/step - loss: 0.9329 - accuracy: 0.5373 - val_loss: 0.8063 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 2/10
3/3 [==============================] - 98s 35s/step - loss: 1.1337 - accuracy: 0.4776 - val_loss: 0.7862 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 3/10
3/3 [==============================] - 97s 35s/step - loss: 1.0535 - accuracy: 0.5373 - val_loss: 0.7310 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 4/10
3/3 [==============================] - 97s 35s/step - loss: 0.9769 - accuracy: 0.5373 - val_loss: 0.7738 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 5/10
3/3 [==============================] - 76s 25s/step - loss: 0.9780 - accuracy: 0.4627 - val_loss: 0.8736 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 6/10
3/3 [==============================] - 76s 25s/step - loss: 1.0474 - accuracy: 0.5373 - val_loss: 1.0165 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 7/10
3/3 [==============================] - 76s 25s/step - loss: 1.3476 - accuracy

In [ ]:
X_train_flattened[:2]

array([[0.09240766, 0.        , 0.        , ..., 0.        , 0.58199954,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.5840363 ,
        0.        ]], dtype=float32)

In [ ]:
y_train

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0]

In [ ]:
X_train

[array([[0.91  , 1.1548, 1.204 ],
        [1.4696, 1.7836, 1.9024]]),
 array([[1.8212, 2.044 , 2.2736],
        [2.4468, 2.4384, 2.4936]]),
 array([[0.9196, 1.082 , 1.2572],
        [1.4072, 1.536 , 1.6532]]),
 array([[1.0572, 1.2852, 1.5392],
        [1.6604, 1.7584, 1.8584]]),
 array([[0.4536, 0.682 , 0.822 ],
        [0.9928, 1.1904, 1.328 ]]),
 array([[1.0908, 1.184 , 1.2936],
        [1.4548, 1.6136, 1.6856]]),
 array([[1.0512, 1.2316, 1.4044],
        [1.5752, 1.6564, 1.7552]]),
 array([[1.0016, 1.2264, 1.4696],
        [1.6096, 1.7156, 1.812 ]]),
 array([[1.0172, 1.1868, 1.3728],
        [1.5048, 1.5908, 1.704 ]]),
 array([[1.03  , 1.3008, 1.56  ],
        [1.694 , 1.7984, 1.8716]]),
 array([[0.828 , 1.0156, 1.1836],
        [1.3476, 1.4952, 1.6092]]),
 array([[1.0296, 1.2248, 1.4224],
        [1.5616, 1.6196, 1.7216]]),
 array([[0.8936, 1.1112, 1.328 ],
        [1.48  , 1.6176, 1.7264]]),
 array([[0.9936, 1.1796, 1.3688],
        [1.5048, 1.5688, 1.6484]]),
 array([[1.0276, 1.2

In [ ]:


# Create an SVM classifier
svm_classifier = SVC(kernel='rbf', random_state=42)

# Train the classifier
svm_classifier.fit(X_train_flattened, y_train)

# Evaluate the model on the test set
svm_test_accuracy = svm_classifier.score(X_test_flattened, y_test)

print(f"SVM Test Accuracy: {svm_test_accuracy}")


SVM Test Accuracy: 0.46875


In [ ]:
# Perform image splitting
X_train, X_test, y_train, y_test = split_dataset(image_list, label_list)

# Resize the images to a common size
resized_X_train = [resize(image, (256, 256)) for image in X_train]
resized_X_test = [resize(image, (256, 256)) for image in X_test]

# Convert the resized images to numpy arrays and flatten
X_train_flattened = np.array(resized_X_train).reshape(len(X_train), -1)
X_test_flattened = np.array(resized_X_test).reshape(len(X_test), -1)

In [ ]:


# Convert the label lists to numpy arrays
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

# Reshape the input data
input_shape = (256, 256, 1)
X_train_reshaped = X_train_flattened.reshape(-1, *input_shape)
X_test_reshaped = X_test_flattened.reshape(-1, *input_shape)

# Normalize the input data
X_train_normalized = X_train_reshaped / 255.0
X_test_normalized = X_test_reshaped / 255.0

# Split the training data into training and validation sets
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_normalized, y_train_np, test_size=0.2, random_state=42
)

# Build the deep learning model
model = tf.keras.models.Sequential() # model object

# Add Layers
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(2, ))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(2))

# Flatten the feature map
model.add(Flatten())

# Add the fully connected layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_final, y_train_final, epochs=5, validation_data=(X_val, y_val))




Epoch 1/5
4/4 [==============================] - 15s 3s/step - loss: 0.6965 - accuracy: 0.4141 - val_loss: 0.6925 - val_accuracy: 0.5600
Epoch 2/5
4/4 [==============================] - 13s 3s/step - loss: 0.6948 - accuracy: 0.4848 - val_loss: 0.6933 - val_accuracy: 0.4400
Epoch 3/5
4/4 [==============================] - 13s 3s/step - loss: 0.6930 - accuracy: 0.5354 - val_loss: 0.6937 - val_accuracy: 0.4400
Epoch 4/5
4/4 [==============================] - 13s 3s/step - loss: 0.6931 - accuracy: 0.5152 - val_loss: 0.6936 - val_accuracy: 0.4400
Epoch 5/5
4/4 [==============================] - 13s 3s/step - loss: 0.6930 - accuracy: 0.5253 - val_loss: 0.6939 - val_accuracy: 0.4400


In [ ]:
y_train_final

array([1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1])

In [ ]:
X_train_final

array([[[[0.00512306],
         [0.00511668],
         [0.0051103 ],
         ...,
         [0.0055088 ],
         [0.00550636],
         [0.00550392]],

        [[0.00511363],
         [0.00510727],
         [0.00510091],
         ...,
         [0.00549604],
         [0.00549365],
         [0.00549126]],

        [[0.00510419],
         [0.00509786],
         [0.00509153],
         ...,
         [0.00548329],
         [0.00548095],
         [0.0054786 ]],

        ...,

        [[0.00515137],
         [0.00514491],
         [0.00513845],
         ...,
         [0.00554707],
         [0.00554448],
         [0.0055419 ]],

        [[0.00514194],
         [0.0051355 ],
         [0.00512906],
         ...,
         [0.00553431],
         [0.00553178],
         [0.00552924]],

        [[0.0051325 ],
         [0.00512609],
         [0.00511968],
         ...,
         [0.00552156],
         [0.00551907],
         [0.00551658]]],


       [[[0.00498989],
         [0.00498172],
         [0.00

In [ ]:
y_train

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0]